In [2]:
from splinter import Browser
from bs4 import BeautifulSoup as bs

import requests

In [39]:
# Set up Splinter
executable_path = {'executable_path':"C:\Program Files (x86)\msedgedriver.exe"}
browser = Browser('edge', **executable_path)

# Url for buying 
url = "https://www.royallepage.ca/en/search/homes/on/toronto/?search_str=Toronto%2C+ON%2C+CAN&csrfmiddlewaretoken=ZEUb1j0hxYEkV3xSu8ezbY169r5o4IQmq6OUHZK6FTChqngtV9EBdplVhZUDePvl&property_type=&house_type=&features=&listing_type=&lat=43.65352400000006&lng=-79.38390699999997&upper_lat=&upper_lng=&lower_lat=&lower_lng=&bypass=&radius=&zoom=&display_type=gallery-view&travel_time=false&travel_time_min=30&travel_time_mode=drive&travel_time_congestion=&da_id=&segment_id=&tier2=False&tier2_proximity=0&address=Toronto&method=homes&address_type=city&city_name=Toronto&prov_code=ON&school_id=&min_price=0&max_price=2500000&min_leaseprice=0&max_leaseprice=5000%2B&beds=0&baths=0&transactionType=SALE&keyword=&sortby="

browser.visit(url)
html = browser.html

soup = bs(html, 'html.parser')

In [40]:
# Go throug first 3 pages to collect data
for x in range(1, 4):
    html = browser.html
    soup = bs(html, 'html.parser')
    links = soup.find_all(class_='address-1')
    
    # Get all address listings for buying    
    house_listings = []
    print('page: ', x, '-----')
    for link in links:
        house_listings.append(link.a['href'])
    browser.find_by_css('.next').click()




page:  1 -----
page:  2 -----
page:  3 -----


In [64]:
# Number of listings
len(house_listings)

46

In [42]:
browser.quit()

In [97]:
# Set up Splinter
executable_path = {'executable_path':"C:\Program Files (x86)\msedgedriver.exe"}
browser = Browser('edge', **executable_path)

# Visit first two links
house_listing_info = []

for i in range(len(house_listings[:2])):
    url = house_listings[i]
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    houses = {} 

    # Gather information on living space
    living_space_section = soup.find('div', class_='property-meta-info')

    # Get exact address
    address = living_space_section.find('div', class_='title--h2 u-no-margins').text


    # Get price
    price = living_space_section.find('span', class_="title title--h1 price").text.strip()

    # Get number of beds
    bed_info = living_space_section.find('div', class_='bed-bath-box__item beds')
    bed_count = bed_info.find('p', class_='bed-bath-box__value').text

    # Get number of baths
    bath_info = living_space_section.find('div', class_='bed-bath-box__item baths')
    bath_count = bath_info.find('p', class_='bed-bath-box__value').text
    
    
    # House features
    building_feature_section = soup.find('div', class_="details-row")
    values = building_feature_section.find_all('span', class_="value")
    
    style_type = values[0].text.strip(":")
    cooling_type = values[-1].text.strip(":")
    heating_fuel = values[-2].text.strip(":")
    heating_type = values[-3].text.strip(":")

    
   
    
    houses['address'] = address
    houses['price'] = price
    houses['number_of_beds'] = bed_count
    houses['number_of_baths'] = bath_count
    houses['style'] = style_type
    houses['cooling_type'] = cooling_type
    houses['heating_fuel'] = heating_fuel
    houses['heating_type'] = heating_type
    
    house_listing_info.append(houses)
    
browser.quit()


In [98]:
house_listing_info

[{'address': '1701-80 Joe Shuster Way, Toronto, ON, M6K 1Y5',
  'price': '$749,900',
  'number_of_beds': '2 Beds',
  'number_of_baths': '1 Bath',
  'style': 'Apartment',
  'cooling_type': 'Gas',
  'heating_fuel': 'Forced Air',
  'heating_type': 'Owned'},
 {'address': '#909 -45 CHARLES ST E, Toronto, Ontario, M4Y 0B8',
  'price': '$749,000',
  'number_of_beds': '1+1 Beds',
  'number_of_baths': '2 Baths',
  'style': 'Apartment',
  'cooling_type': 'Central air conditioning',
  'heating_fuel': 'Natural gas',
  'heating_type': 'Forced air'}]